In [1]:
from dichasus_cf0x import training_set
import tensorflow as tf
import numpy as np
import tqdm

2023-10-23 07:22:25.566590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 07:22:26.373907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-23 07:22:27.224276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 07:22:27.248840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
# Count number of datapoints in training set (for progress bar)
TOTAL_DATAPOINTS = sum(1 for _ in training_set)

In [3]:
ANTENNAS_PER_ROW = tf.shape(training_set.take(1).get_single_element()[0])[2].numpy()

In [4]:
estimated_toas = np.load("results/estimated_toas.npy")

In [5]:
def get_unitary_rootmusic_estimator(chunksize = 32, shed_coeff_ratio = 0):
    I = np.eye(chunksize // 2)
    J = np.flip(np.eye(chunksize // 2), axis = -1)
    Q = np.asmatrix(np.block([[I, 1.0j * I], [J, -1.0j * J]]) / np.sqrt(2))
    
    def unitary_rootmusic(R):
        assert(len(R) == chunksize)
        C = np.real(Q.H @ R @ Q)
    
        eig_val, eig_vec = np.linalg.eigh(C)
        eig_val = eig_val[::-1]
        eig_vec = eig_vec[:,::-1]

        source_count = 1
        En = eig_vec[:,source_count:]
        ENSQ = Q @ En @ En.T @ Q.H
    
        coeffs = np.asarray([np.trace(ENSQ, offset = diag) for diag in range(1, len(R))])
        coeffs = coeffs[:int(len(coeffs) * (1 - shed_coeff_ratio))]

        # Remove some of the smaller noise coefficients, trade accuracy for speed
        coeffs = np.hstack((coeffs[::-1], np.trace(ENSQ), coeffs.conj()))
        roots = np.roots(coeffs)
        roots = roots[abs(roots) < 1.0]
        largest_root = np.argmax(1 / (1.0 - np.abs(roots)))
        
        return np.angle(roots[largest_root])

    return unitary_rootmusic

In [6]:
umusic = get_unitary_rootmusic_estimator(ANTENNAS_PER_ROW)

estimated_aoas = []
for index, data in enumerate(tqdm.tqdm(training_set, total = TOTAL_DATAPOINTS)):
    csi, pos, time = data[0], data[1], data[2]

    # Extract channel coefficients at the arrival time of the LoS component
    transform_vectors = np.exp(1.0j * 2 * np.pi * np.outer(estimated_toas[index], np.arange(-np.shape(csi)[-1] // 2, np.shape(csi)[-1] // 2)) / np.shape(csi)[-1])
    csi_at_los = np.einsum("arms,as->arm", csi, transform_vectors)
    R = np.einsum("arm,arn->amn", csi_at_los, np.conj(csi_at_los))

    estimated_aoas.append([np.arcsin(umusic(R[array]) / np.pi) for array in range(len(R))])

100%|██████████| 20997/20997 [00:26<00:00, 789.38it/s]


In [7]:
estimated_aoas = np.asarray(estimated_aoas)

In [8]:
np.save("results/estimated_aoas.npy", estimated_aoas)